<a href="https://colab.research.google.com/github/scudilio/FIAP/blob/main/RN_AULA_14_LSTM_%2B_ANALISE_SENTIMENTO_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AULA 11 - LSTM: ANÁLISE DE SENTIMENTO

**Objetivo:**
Treinar uma rede LSTM para realizar análise de sentimento em um conjunto de dados de reviews de filmes, classificando-os como positivos ou negativos.

Passos:
1. Carregar e pré-processar os dados de análise de sentimento.
2. Criar a rede LSTM.
3. Treinar o modelo.
4. Avaliar a performance.


##1. IMPORTANDO AS BIBLIOTECAS

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical


## 2. Carregar e Pré-processar o Conjunto de Dados
Vamos usar o conjunto de dados IMDB de reviews de filmes que já está incluído no Keras. Ele contém 25.000 reviews de filmes rotulados como positivos ou negativos.

In [ ]:
# Carregar o dataset IMDB
max_features = 10000  # Número máximo de palavras a considerar
maxlen = 100  # Tamanho máximo da sequência (número de palavras)

# Carregar os dados de treino e teste do dataset IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Padding das sequências para garantir que tenham o mesmo tamanho
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


In [ ]:
x_train

array([[1415,   33,    6, ...,   19,  178,   32],
       [ 163,   11, 3215, ...,   16,  145,   95],
       [1301,    4, 1873, ...,    7,  129,  113],
       ...,
       [  11,    6, 4065, ...,    4, 3586,    2],
       [ 100, 2198,    8, ...,   12,    9,   23],
       [  78, 1099,   17, ...,  204,  131,    9]], dtype=int32)

## 3. Criar o Modelo LSTM
Agora, vamos criar um modelo sequencial com uma camada de Embedding, seguida de uma camada LSTM, e terminar com uma camada densa para a classificação binária (positivo ou negativo).

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length = maxlen))  # Embedding para vetorizar as palavras
model.add(LSTM(128, return_sequences = True, dropout=0.2, recurrent_dropout=0.2))  # Primeira camada LSTM
model.add(LSTM(128, return_sequences = True, dropout=0.2, recurrent_dropout=0.2))  # Segunda camada LSTM

model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))  # Terceira camada LSTM com menos unidades
model.add(Dense(1, activation='sigmoid'))  # Camada final para a classificação binária

# Compilando o modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


##Explicação do Código:
* Embedding Layer: Converte as palavras em vetores de tamanho fixo. Cada palavra é mapeada para um vetor de 128 dimensões.
* LSTM Layer: A camada LSTM processa sequências de palavras, permitindo que o modelo mantenha informações importantes ao longo da sequência.
* Dense Layer: A camada densa final usa a função de ativação sigmoide para prever a probabilidade de uma review ser positiva ou negativa.
* Binary Crossentropy: Como estamos realizando uma classificação binária, usamos a função de perda binary_crossentropy.
* Adam Optimizer: Um algoritmo de otimização eficiente e amplamente utilizado.


Conclusão:
Este modelo básico de LSTM pode ser treinado para realizar análise de sentimentos em dados textuais. Ajustes adicionais podem incluir a adição de mais camadas LSTM, regulação com Dropout, e treinamento com mais épocas (epochs) para melhorar a precisão.

##4. Treinar o Modelo
Agora que o modelo foi criado, podemos treiná-lo nos dados de treino.

In [ ]:
batch_size = 32
epochs = 3

# Treinando o modelo
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=2)


Epoch 1/3
782/782 - 454s - 581ms/step - accuracy: 0.7715 - loss: 0.4734 - val_accuracy: 0.8408 - val_loss: 0.3604
Epoch 2/3
782/782 - 493s - 630ms/step - accuracy: 0.8665 - loss: 0.3217 - val_accuracy: 0.8289 - val_loss: 0.3875
Epoch 3/3
782/782 - 487s - 623ms/step - accuracy: 0.8927 - loss: 0.2708 - val_accuracy: 0.8448 - val_loss: 0.3695


## 5. Avaliar o Modelo
Após o treinamento, vamos avaliar a performance do modelo nos dados de teste.

In [ ]:
# Avaliação do modelo
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)


## Analisando uma avaliacao e vendo sua previsao

In [ ]:
# Carregar o dicionário de palavras
word_index = imdb.get_word_index()

# Função para converter uma sequência numérica de volta para uma string de palavras
def decode_review(sequence):
    reverse_word_index = {value: key for (key, value) in word_index.items()}
    # O índice 0, 1 e 2 são reservados para preenchimento, início da sequência, etc.
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in sequence])

# Exibir uma avaliação específica (primeira no conjunto de teste, por exemplo)
print(decode_review(x_test[0]))

In [ ]:
# Prever o sentimento da primeira avaliação do conjunto de teste
review_to_predict = x_test[0]

# Fazer a previsão com o modelo treinado
prediction = model.predict(np.array([review_to_predict]))

# Exibir o resultado da previsão
if prediction[0] > 0.5:
    print("Positivo")
else:
    print("Negativo")


In [ ]:
prediction

array([[0.12211445]], dtype=float32)

## Analisando uma avaliação manual

In [ ]:
# Texto de exemplo para fazer a previsão
new_review = "This movie was fantastic! I loved the acting and the plot was very interesting."

# Tokenizar e converter o texto para sequência numérica
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts([new_review])
sequence = tokenizer.texts_to_sequences([new_review])

# Padronizar a sequência para ter o mesmo tamanho usado no treinamento
sequence_padded = pad_sequences(sequence, maxlen=maxlen)

# Fazer a previsão com o modelo treinado
prediction = model.predict(sequence_padded)

# Exibir o resultado da previsão
if prediction[0] > 0.5:
    print("Positivo")
else:
    print("Negativo")


In [ ]:
print(y_test[0])  # Mostra o rótulo da primeira avaliação do conjunto de teste


0
